In [ ]:
!pip install openai google-genai pandas python-dotenv
import json
from pathlib import Path
from typing import Dict, Iterable, List, Optional
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from openai import OpenAI
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) 


In [ ]:

schema = {
    "type": "object",
    "properties": {
        "validation_present": {
            "type": "boolean"
        },
        "evidence_sources": {
            "type": "array",
            "items": {
                "type": "string",
                "enum": ["pipeline", "description", "comments"]
            }
        },
        "validation_type": {
            "type": "string",
            "enum": [
                "benchmark",
                "profiling",
                "static-analysis",
                "anecdotal",
                "load/canary",
                "none"
            ]
        },
        "validation_description": {
            "type": "string"
        },
        "pipeline_signal": {
            "type": "string"
        },
        "description_signal": {
            "type": "string"
        },
        "comment_signal": {
            "type": "string"
        }
    },
    "required": [
        "validation_present",
        "evidence_sources",
        "validation_type",
        "validation_description",
        "pipeline_signal",
        "description_signal",
        "comment_signal"
    ]
}

schema_openai = {
        "type": "object",
        "properties": {
            "validation_present": {"type": "boolean"},
            "evidence_sources": {
                "type": "array",
                "items": {
                    "type": "string",
                    "enum": ["pipeline", "description", "comments"]
                }
            },
            "validation_type": {
                "type": "string",
                "enum": [
                    "benchmark",
                    "profiling",
                    "static-analysis",
                    "anecdotal",
                    "load/canary",
                    "none"
                ]
            },
            "validation_description": {"type": "string"},
            "pipeline_signal": {"type": "string"},
            "description_signal": {"type": "string"},
            "comment_signal": {"type": "string"}
        },
        "required": [
            "validation_present",
            "evidence_sources",
            "validation_type",
            "validation_description",
            "pipeline_signal",
            "description_signal",
            "comment_signal"
        ],
        "additionalProperties": False,
        }

In [ ]:

def run_llm(prompt: str,user_prompt: str, type: str = "openai", model: str = "gpt-5.1-2025-11-13") -> str:
    """Call local OLLAMA."""
    LLM_BASE_URL = None
    LLM_API_KEY = os.environ.get("OPENAI_API_KEY")
    llm_client = OpenAI(base_url=LLM_BASE_URL, api_key=LLM_API_KEY)
    
    if(type == "openai"):
        model = "gpt-5.1-2025-11-13"
        print("Running OPENAI, model:", model)
  
        messages = [
            {"role": "developer", "content": prompt},
            {"role": "user", "content": user_prompt},
        ]
        

        r = llm_client.chat.completions.create(
            model=model,
            temperature=0.0,
            messages=messages,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "validation_schema",
                    "schema": schema_openai,
                    "strict": True,
                },
            },
        )

        result = r.choices[0].message.content.strip()
        return result
    elif(type == "gemini"):
        
        model = "models/gemini-pro-latest"
        print("Running GEMINI, model:", model)
        
        GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
        client = genai.Client(api_key=GEMINI_API_KEY)

        
        config = types.GenerateContentConfig(
            temperature=0.0,
            response_mime_type="application/json",
            response_schema=schema,
            # thinking_config=types.ThinkingConfig(
            #     thinking_level=types.ThinkingLevel.HIGH
            # )
        )
        
        messages = [
            prompt,
            user_prompt,
        ]
        
        response = client.models.generate_content(
            model=model,
            contents=messages,
            config=config,
        )
        return response.text
    

In [ ]:
def find_datasets_dir(start: Optional[Path] = None) -> Path:
    start = start or Path.cwd()
    for path in (start, *start.parents):
        candidate = path / "datasets"
        if candidate.exists():
            return candidate
    raise FileNotFoundError(f"Could not find 'datasets' directory from {start}")


DATASETS_DIR = find_datasets_dir()
PROJECT_ROOT = DATASETS_DIR.parent

In [ ]:
def extract_json(text: str) -> Dict:
    """Best-effort JSON extraction from model output."""
    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1 and end > start:
        snippet = text[start : end + 1]
        try:
            return json.loads(snippet)
        except json.JSONDecodeError:
            pass
    return {}

def truncate(text: str, limit: int = 10000) -> str:
    return text if len(text) <= limit else text[:limit] + "...[truncated]"

In [ ]:
def load_pr_core(prefix: str) -> pd.DataFrame:
    commits = pd.read_parquet(
        DATASETS_DIR / f"{prefix}_pr" / f"{prefix}_pr_commits.parquet"
    )
    return commits.drop_duplicates("pr_id").set_index("pr_id")


def collect_comments(prefix: str, pr_id: int) -> List[str]:
    issue = pd.read_parquet(
        DATASETS_DIR / f"{prefix}_pr" / f"{prefix}_pr_issue_comments.parquet"
    )
    review = pd.read_parquet(
        DATASETS_DIR / f"{prefix}_pr" / f"{prefix}_pr_review_comments.parquet"
    )
    texts = []
    for df in (issue, review):
        subset = df[df["pr_id"] == pr_id]
        texts.extend(subset["body"].dropna().tolist())
    return texts


def collect_pipeline_names(prefix: str, pr_id: int) -> List[str]:
    workflows = pd.read_parquet(
        DATASETS_DIR / f"{prefix}_pr" / f"{prefix}_pr_workflow_runs.parquet"
    )
    subset = workflows[workflows["pr_id"] == pr_id]
    return sorted(subset["workflow_name"].dropna().unique().tolist())


In [ ]:
def analyze_pr(
    prefix: str, pr_id: int, author_type: str, pr_core: pd.DataFrame
) -> tuple[Dict, Dict]:
    row = pr_core.loc[pr_id]
    pipeline_names = collect_pipeline_names(prefix, pr_id)
    comments = collect_comments(prefix, pr_id)
    description = (row.get("pr_description") or "").strip()
    code_diff = (row.get("patch") or "").strip()
    
    DEVELOPER_PROMPT_TEMPLATE = """
    You are a classifier for performance validation evidence in GitHub Pull Requests (PRs).

Your job is to decide whether there is explicit performance validation, and if so,
to classify it into EXACTLY ONE validation_type category based on the definitions below.

First, carefully read and internalize these validation_type categories:

1. Benchmark-Based Validation (validation_type="benchmark")
Definition:
The PR validates performance by running benchmark tests—either existing unit tests or newly
added benchmark/microbenchmark tests. It includes explicit before-and-after comparisons such as
runtime, throughput, memory, CPU usage, or any quantitative metric collected from tests.

2. Profiling-Based Validation (validation_type="profiling")
Definition:
The PR uses profiling tools to validate performance, typically capturing stack samples,
CPU hotspots, flamegraphs, or function-level timings. Evidence includes profiling outputs
before and after the change.

3. Static-Analysis-Based Validation (validation_type="static-analysis")
Definition:
The PR argues for performance improvement through static reasoning about the code—algorithmic
complexity, data-structure changes, loop bounds, allocation count reduction, etc.—without
providing runtime or profiling data.

4. Anecdotal or Informal Local Testing (validation_type="anecdotal")
Definition:
The PR claims that performance is improved based on local testing, intuition, or manual
observation (for example, “this feels faster on my machine”, “latency looks better now”),
but provides no quantitative metrics, no profiling output, and no detailed static-analysis
justification.

5. Load/Canary-Based Validation (validation_type="load/canary")
Definition:
The PR validates performance by running load tests, stress tests, or canary/phased rollouts
under real or synthetic traffic. Evidence includes references to load-testing tools,
canary deployments, A/B tests, gradual rollout with monitoring, or production metrics under load.

6. No Validation Evidence (validation_type="none")
Definition:
The PR provides no explicit evidence of performance validation. It may mention optimization intent,
but there are no benchmarks, no profiling, no static reasoning, no local-testing claims, and no
load/canary validation described.

Decision Rules:

1) First decide if performance validation is explicitly present (validation_present).
   - Set validation_present = TRUE only when the PR explicitly shows some form of validation evidence:
     benchmarks/microbenchmarks, profiling traces, static performance reasoning, load/canary rollout,
     or explicit local testing statements.
   - Do NOT infer validation from performance intent alone.

2) If validation_present = FALSE:
   - Set validation_type = "none".
   - Set evidence_sources = [].
   - Set validation_description to explain the absence of validation.
   - Never assign any other validation_type.

3) If validation_present = TRUE:
   - Choose exactly ONE non-"none" validation_type with this priority:
     benchmark > profiling > load/canary > static-analysis > anecdotal

4) evidence_sources must list where the validation is explicitly mentioned:
   - "pipeline", "description", "comments"

5) When validation_type is "benchmark", "profiling", or "load/canary",
   mention the metrics used (latency, throughput, memory, CPU, etc.).

You must ALWAYS return STRICT JSON with exactly these keys:

validation_present, evidence_sources, validation_type,
validation_description, pipeline_signal,
description_signal, comment_signal.

No extra commentary. No markdown.
No explanations.
    """
    
    USER_PROMPT_TEMPLATE = """
    Classify the following PR strictly using the rules and definitions from the system.

    PIPELINES:
    {pipeline_names}

    DESCRIPTION:
    {description}

    COMMENTS:
    {comments}
    
    CODE DIFF:
    {code_diff}
    """
    
   
    empty_record = {
        "pr_id": pr_id,
        "author_type": author_type,
        "repo": f"{row.get('repo_owner')}/{row.get('repo_name')}",
        "pr_number": row.get("pr_number"),
        "pr_title": row.get("pr_title"),
        "pipeline_names": pipeline_names,
        "validation_present": False,
        "evidence_sources": [],
        "validation_type": "none",
        "validation_description": "No validation evidence",
        "pipeline_signal": "",
        "description_signal": "",
        "comment_signal": "",
    }

    if not pipeline_names and not description and not comments:
        return empty_record, empty_record

    prompt = USER_PROMPT_TEMPLATE.format(
        pipeline_names="- " + "- ".join(pipeline_names) if pipeline_names else "None",
        description=truncate(description) if description else "None",
        comments="- " + "- ".join(truncate(" | ".join(comments)).split(" | ")) if comments else "None",
        code_diff=truncate(code_diff) if code_diff else "None",
    )
    
    developer_prompt = DEVELOPER_PROMPT_TEMPLATE

    raw_openai = ""
    raw_gemini = ""
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = {
            "openai": executor.submit(
                run_llm, prompt=developer_prompt, user_prompt=prompt, type="openai"
            ),
            "gemini": executor.submit(
                run_llm, prompt=developer_prompt, user_prompt=prompt, type="gemini"
            ),
        }
        for name, fut in futures.items():
            try:
                result = fut.result()
            except Exception as exc:
                result = ""
                print(f"{name} model call failed for PR {pr_id}: {exc}")
            if name == "openai":
                raw_openai = result
            else:
                raw_gemini = result

    parsed_openai = extract_json(raw_openai) or {}
    parsed_gemini = extract_json(raw_gemini) or {}

    def build_result(parsed: Dict):
        evidence_sources = parsed.get("evidence_sources") or []
        if isinstance(evidence_sources, (tuple, list)):
            evidence_sources = list(evidence_sources)
        
        return {
            "pr_id": pr_id,
            "author_type": author_type,
            "repo": f"{row.get('repo_owner')}/{row.get('repo_name')}",
            "pr_number": row.get("pr_number"),
            "pr_title": row.get("pr_title"),
            "pipeline_names": pipeline_names,
            "validation_present": parsed.get("validation_present"),
            "evidence_sources": evidence_sources,
            "validation_type": parsed.get("validation_type"),
            "validation_description": parsed.get("validation_description"),
            "pipeline_signal": parsed.get("pipeline_signal"),
            "description_signal": parsed.get("description_signal"),
            "comment_signal": parsed.get("comment_signal"),
        }

    return build_result(parsed_openai), build_result(parsed_gemini)


In [ ]:
def pr_ids_from_commits(prefix: str, limit: Optional[int] = None) -> Iterable[int]:
    commits = pd.read_parquet(
        DATASETS_DIR / f"{prefix}_pr" / f"{prefix}_pr_commits.parquet"
    )
    pr_ids = sorted(commits["pr_id"].dropna().astype(int).unique().tolist())
    return pr_ids if limit is None else pr_ids[:limit]


In [ ]:
out_dir = PROJECT_ROOT / "RQ3"
out_dir.mkdir(exist_ok=True, parents=True)
out_path_openai = out_dir / "rq3_validation_evidence_openai.parquet"
out_path_gemini = out_dir / "rq3_validation_evidence_gemini.parquet"

records_openai = []
records_gemini = []
ai_core = load_pr_core("ai")
human_core = load_pr_core("human")

limit = None

def save_partial(records, out_path):
    df_tmp = pd.DataFrame(records)
    df_tmp.to_parquet(out_path, index=False)
    print(f"[partial save] Saved {len(df_tmp)} rows to {out_path}")

ai_ids = list(pr_ids_from_commits("ai", limit=limit))
human_ids = list(pr_ids_from_commits("human", limit=limit))
print(f"Processing {len(ai_ids)} AI PRs and {len(human_ids)} human PRs (first {limit} each).")

# ============================
# Process AI PRs
# ============================
for idx, pr_id in enumerate(ai_ids, 1):
    print(f"Processing AI PR {idx}/{len(ai_ids)}: {pr_id}")
    try:
        openai, gemini = analyze_pr("ai", pr_id, "ai_agent", ai_core)
        records_openai.append(openai)
        records_gemini.append(gemini)
    except Exception as exc:
        empty = {
            "pr_id": pr_id,
            "author_type": "human",
            "repo": "",
            "pr_number": None,
            "pr_title": "",
            "pipeline_names": [],
            "validation_present": None,
            "evidence_sources": [],
            "validation_type": "error",
            "validation_description": f"error: {exc}",
            "pipeline_signal": "",
            "description_signal": "",
            "comment_signal": "",
        }
        records_openai.append(empty)
        records_gemini.append(empty)

    # ---- SAVE EVERY 10 ----
    if len(records_openai) % 10 == 0:
        save_partial(records_openai, out_path_openai)
        save_partial(records_gemini, out_path_gemini)

# ============================
# Process Human PRs
# ============================
for idx, pr_id in enumerate(human_ids, 1):
    print(f"Processing human PR {idx}/{len(human_ids)}: {pr_id}")
    try:
        openai, gemini = analyze_pr("human", pr_id, "human", human_core)
        records_openai.append(openai)
        records_gemini.append(gemini)
    except Exception as exc:
        empty = {
            "pr_id": pr_id,
            "author_type": "human",
            "repo": "",
            "pr_number": None,
            "pr_title": "",
            "pipeline_names": [],
            "validation_present": None,
            "evidence_sources": [],
            "validation_type": "error",
            "validation_description": f"error: {exc}",
            "pipeline_signal": "",
            "description_signal": "",
            "comment_signal": "",
        }
        records_openai.append(empty)
        records_gemini.append(empty)

    # ---- SAVE EVERY 20 ----
    if len(records_openai) % 10 == 0:
        save_partial(records_openai, out_path_openai)
        save_partial(records_gemini, out_path_gemini)

# ============================
# Final save
# ============================
df_open_ai = pd.DataFrame(records_openai)
df_gemini = pd.DataFrame(records_gemini)
df_open_ai.to_parquet(out_path_openai, index=False)
df_gemini.to_parquet(out_path_gemini, index=False)
print(f"Saved FINAL OPENAI {len(df_open_ai)} rows to {out_path_openai}")
print(f"Saved FINAL GEMINI {len(df_gemini)} rows to {out_path_gemini}")

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", True)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

data_temp = pd.read_parquet(out_path_openai)
data_temp.tail(20)

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", True)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

data_temp = pd.read_parquet(out_path_gemini)
data_temp.tail(20)